In [7]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
#from matplotlib.pyplot import plt
import numpy as np
import pandas as pd
import time
import os
import copy
from functools import partial

Import/Load Data 


In [6]:
model_save_path = '/home/wdougall/models/rnsa-densenet.pt'

#change to chest xrays transfer from rnsa to chest xray

#maybe not

train_img_dir = '/home/wdougall/assets/rnsa-pneumonia/train-images'
annotations_file_path = '/home/wdougall/assets/rnsa-pneumonia/stage_2_train_labels.csv.zip'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# transform = torchvision.transforms.Compose([
#     torchvision.transforms.Resize(512),
#     torchvision.transforms.CenterCrop(448),
#     torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize(mean, std),
# ])

# label_transform = torchvision.transforms.Compose([
#     partial(torch.tensor, dtype=torch.float),
#     partial(torch.unsqueeze, dim=0),
# ])
    
# test_dataset = rnsa.RNSADataset(train_img_dir, annotations_file_path, test_ix, transform, label_transform)
# test_data_loader = rnsa.get_data_loader(test_dataset, batch_size=32)

Import Model

In [ ]:
#Try with pretrained model 
#model = models.densenet121(pretrained=True, weights= "DEFAULT")
model, best_loss, best_acc, lr = rnsa.load_checkpoint(model_save_path, 'cpu')

print('checkpoint best loss: {:.4}'.format(best_loss))
print('checkpoint best acc: {:.4}'.format(best_acc))

Extract Features 

In [8]:
import sklearn
from sklearn import cross_validation, grid_search
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.externals import joblib

def train_svm(features, labels, model_save_path):
    # label_df = pd.read_csv(annotations_file_path).groupby('patientId').first().reset_index()

    # X_train, X_test = train_test_split(label_df, test_size=.2, stratify=label_df.Target, random_state=99)
    # X_val, X_test, = train_test_split(X_test, test_size=.4, stratify=X_test.Target, random_state=99)
    #train_ix, val_ix, test_ix = X_train.index.tolist(), X_val.index.tolist(), (X_test.index.tolist())
    
    #print('train {:,} - validate {:,} - test {:,}'.format(len(train_ix), len(val_ix), len(test_ix)))
    
    # Single channel mean & standard deviation.
    mean = [0.5]
    std = [0.225]

    params = [{"kernel": ["linear"],
        "C" : [1,5, 10, 50, 100, 1000]}, 
        {"kernel": ["rbf"],
        "C" : [1,5, 10, 50, 100, 1000],
        "gamma": [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
        }]

    svm = SVC(probability = True)

    clf = grid_search.GridSearchCV(svm, params, cv = 10, n_jobs = 4, verbose = 3)

    #PAINPOINT 


    #we want to fit the SVM on the extracted features from the images,
    #this will require a new train test split of image features and classes 
    clf.fit(X_train, y_train) 

    if os.path.exists(model_save_path):
        joblib.dump(clf.best_estimator_, model_save_path)

    else: 
        print("Check model save path, SVM save failed")

    
    print('SVM: Optimal Parameters: ', str(clf.best_params_))

    y_pred = clf.predict(X_test)

    labels = sorted(list(set()))




ModuleNotFoundError: No module named 'sklearn'

Train SVM/SVC on Features from PT/CKPT File 

Predict 

Validate